# 不分词，字向量法训练分类

@PM


### 准备工作： 连接数据库并获取各类文本数据到列表


In [1]:
import sqlite3
conn = sqlite3.connect("../data/database/texts.db")
print ('**Opened database successfully**\n')
c = conn.cursor()
ci_data=[]
cursor = c.execute("SELECT * from ci order by id asc limit 2000")
for row in cursor:
    ci_data.append(row)
print("读取的词数据样例：")
print(ci_data[-2:])

poet_data = []
cursor=c.execute("SELECT * FROM poet order by id asc limit 2000")
for row in cursor:
    poet_data.append(row)
print("\n读取的诗数据样例：")
print(poet_data[-2:])

classical_data = []
cursor=c.execute("SELECT * FROM classical order by id asc limit 2000")
for row in cursor:
    classical_data.append(row)
print("\n读取的文言文数据样例：")
print(classical_data[-2:])

journal_data = []
cursor=c.execute("SELECT * FROM journal order by id asc limit 2000")
for row in cursor:
    journal_data.append(row)
print("\n读取的期刊数据样例：")
print(journal_data[:2])
    
news_data = []
cursor=c.execute("SELECT * FROM news order by id asc limit 2000")
for row in cursor:
    news_data.append(row)
print("\n读取的新闻数据样例：")
print(news_data[:2])

**Opened database successfully**

读取的词数据样例：
[(50001999, '落日衔山，行云载雨俄鸣。 一顷新荷，坐间疑是秋声。 烟波醉客，见快哉、风恼娉婷。 香和清点，为人吹在衣襟。 珠佩欢言，放船且向前汀。 绿伞红幢，自从天汉相迎。 飞鸥独落，芦边对、几朵繁英。 侑觞人唱，乍闻应似湘灵。', 'ci', 1999), (50002000, '多幸春来云雨少。 且教月与花相照。 清色真香庭院悄。 前事杳。 还嗟此景何时了。 莫道难逢开口笑。 夜游须趁人年少。 光泛雕栏寒料峭。 迂步绕。 不劳秉烛壶天晓。  >>  词牌介绍', 'ci', 2000)]

读取的诗数据样例：
[(40001999, '朱明日盛残花卉，琼苑争游諠帝里。 宝马香车去复来，几许人心欢不已。 金明水上浮仙岛，画舸龙舟非草草。 世宁清静验如然，老者携小少随老。 匼匝烟云杨柳岸，罗绮纵横长不断。 五谷丰登顺四时，亿兆歌谣绝愁叹。 康哉阗咽芳林下，一看难醻千万价。 千平听在乐声中，比屋可封民自化。 ', 'poet', '5697307b-2273-42f3-b6fb-b4f0006e0009'), (40002000, '五粒青松护翠苔，石门岑寂断纤埃。 水浮花片知仙路，风递鸾声认啸台。 桐井晓寒千乳敛，茗园春嫩一旗开。 驰烟未勒山亭字，可是英灵许再来。 ', 'poet', 'dad91d22-4b8a-4c04-a0d5-8f7ca8aff4de')]

读取的文言文数据样例：
[(10002000, '    君子将营宫室。宗庙为先，厩库为次，居室为后。凡家造，祭器为先，牺赋 为次，养器为后。无田禄者不设祭器；有田禄者，先为祭服。君子虽贫，不粥祭 器；虽寒，不衣祭服；为宫室，不斩于丘木。\n', 'classical', 'c_list_10.txt'), (10002001, '    大夫士去国，祭器不逾竟。大夫寓祭器于大夫，士寓祭器于士。大夫去国， 逾竟，为坛位乡国而哭。素衣、素裳、素冠、彻缘、?屦、素?蔑，乘髦马。不 蚤{髟前}。不祭食，不说人以无罪；妇人不当御。三月而复服。\n', 'classical', 'c_list_10.txt')]

读取的期刊数据样例：
[(20000001, '作者程苏东, 北京大学中文系讲师 (北京10087

In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
classifier = MultinomialNB()
tv = TfidfVectorizer(analyzer='char',max_features=5000, lowercase = False)


def train_NBayes(x_train, x_test, y_train, y_test):
    #模型训练
    classifier.fit(vectorizer.transform(x_train), y_train)
    #使用训练好的模型进行预测
    print("朴素贝叶斯得分："+str(classifier.score(vectorizer.transform(x_test), y_test)))
    
def train_TfIdf(x_train, x_test, y_train, y_test):
    tv.fit(x_train)
    #模型训练
    classifier.fit(tv.transform(x_train), y_train)
    #利用训练好的模型测试
    print("TF-IDF得分："+str(classifier.score(tv.transform(x_test), y_test)))
def train_nGram(x_train, x_test, y_train, y_test,n):
    #转向量
    tv_2gram = TfidfVectorizer(analyzer='char', max_features=5000, ngram_range=(1,n),lowercase = False)
    tv_2gram.fit(x_train)
    #训练模型
    clf_2gram = MultinomialNB()
    clf_2gram.fit(tv_2gram.transform(x_train), y_train)
    #预测
    print("ngram,n为"+str(n)+"时得分："+str(clf_2gram.score(tv_2gram.transform(x_test), y_test)))

## ⅠBasic方案：对原文本拆分成单字
### 1. 暴力拆字


In [2]:
data = ci_data + poet_data + classical_data + journal_data + news_data
data_list = []
for line in data:
    data_list.append(list(line))

print(data_list[6000])
for item in data_list:
    item[1] = ' '.join(list(item[1]))
print(data_list[6000])
print(len(data_list))





[20000001, '作者程苏东, 北京大学中文系讲师 (北京100871) 。\n', 'modern', 'j_list_1.txt']
[20000001, '作 者 程 苏 东 ,   北 京 大 学 中 文 系 讲 师   ( 北 京 1 0 0 8 7 1 )   。 \n', 'modern', 'j_list_1.txt']
10000


### 2. 建立训练集、测试集

In [3]:
train_data=[]
train_target=[]
for i in range(0,10000):
    train_data.append(data_list[i][1])
    train_target.append(data_list[i][2])

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, train_target,test_size=0.4,random_state=0)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(analyzer='char')
vector = cv.fit_transform(x_train)
vector.todense()

matrix([[ 153,    0,    0, ...,    0,    0,    0],
        [  13,    0,    0, ...,    0,    0,    0],
        [ 128,    0,    0, ...,    0,    0,    0],
        ...,
        [  32,    0,    0, ...,    0,    0,    0],
        [1085,    0,    0, ...,    0,    0,    0],
        [  12,    0,    0, ...,    0,    0,    0]], dtype=int64)

In [56]:
# cv.vocabulary_

In [12]:
vectorizer = CountVectorizer(analyzer='char', max_features=5000)
vectorizer.fit(x_train)
print(x_train[:3])

['。 因 此 ,   基 层 治 理 能 力 制 度 建 设 也 与 村 庄 社 会 基 础 有 密 切 的 关 系 。 杨 念 群 认 为 ,   与 城 市 中 的 社 会 生 活 相 比 ,   农 村 社 会 的 程 式 化 和 模 式 化 程 度 是 很 低 的 ,   实 际 上 缺 少 一 成 不 变 的 正 式 程 序 和 正 式 规 则 。 在 许 多 情 况 下 ,   即 使 存 在 这 样 的 程 序 和 规 则 ,   有 时 也 不 会 真 正 起 作 用 ; 相 反 ,   一 些 重 要 而 敏 感 问 题 的 解 决 ,   往 往 采 取 非 正 式 的 方 式 或 随 机 处 理 的 弹 性 手 段 \n', '\u3000 \u3000 众 趋 明 所 避 ， 时 弃 道 犹 存 。   \n', '桐 轩 潇 洒 远 尘 机 ， 伯 始 英 风 世 不 衰 。   千 亩 自 封 轻 渭 叟 ， 五 经 同 拜 重 宣 尼 。   衣 无 常 主 门 多 义 ， 仓 有 余 粮 俗 共 熙 。   仁 智 眼 前 从 所 乐 ， 利 名 身 外 更 何 思 。   琴 清 善 召 云 中 鹤 ， 床 稳 高 蓍 叶 上 龟 。   茶 煮 玉 泉 僧 至 日 ， 酒 𫇴 金 蚁 菊 芳 时 。   往 来 虚 席 宾 朋 盛 ， 朝 暮 过 堂 子 弟 奇 。   万 卷 诗 书 归 腹 箧 ， 地 仙 踪 迹 少 人 知 。  ']


In [25]:
print("暴力拆字方法的训练结果：")
train_NBayes(x_train, x_test, y_train, y_test)
train_TfIdf(x_train, x_test, y_train, y_test)
train_nGram(x_train, x_test, y_train, y_test,2)

暴力拆字方法的训练结果：
朴素贝叶斯得分：0.87575
TF-IDF得分：0.8175
ngram,n为2时得分：0.879


## Pro: 清理时去掉符号（利用正则表达式）

In [40]:
import re

train_data_pro = []
for paragraph in train_data:
    train_data_pro.append(' '.join(list((re.sub("[\s+\.\!\/_,$%^*)(+\"\']+|[+——！，。？、~@#￥%……&*（）]", "",paragraph)))))


print("去除符号成功：\n样例：【"+train_data[6000]+"】=========>【"+train_data_pro[6000]+"】")

去除符号成功：
样例：【作 者 程 苏 东 ,   北 京 大 学 中 文 系 讲 师   ( 北 京 1 0 0 8 7 1 )   。 
】=========>【作 者 程 苏 东 北 京 大 学 中 文 系 讲 师 北 京 1 0 0 8 7 1】


In [42]:
x_train_pro, x_test_pro, y_train, y_test = train_test_split(train_data_pro, train_target,test_size=0.4,random_state=0)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(analyzer='char')
vector_pro = cv.fit_transform(x_train_pro)
vector_pro.todense()
vectorizer_pro = CountVectorizer(analyzer='char', max_features=5000)
vectorizer_pro.fit(x_train_pro)
print(x_train_pro[:3])

['因 此 基 层 治 理 能 力 制 度 建 设 也 与 村 庄 社 会 基 础 有 密 切 的 关 系 杨 念 群 认 为 与 城 市 中 的 社 会 生 活 相 比 农 村 社 会 的 程 式 化 和 模 式 化 程 度 是 很 低 的 实 际 上 缺 少 一 成 不 变 的 正 式 程 序 和 正 式 规 则 在 许 多 情 况 下 即 使 存 在 这 样 的 程 序 和 规 则 有 时 也 不 会 真 正 起 作 用 ; 相 反 一 些 重 要 而 敏 感 问 题 的 解 决 往 往 采 取 非 正 式 的 方 式 或 随 机 处 理 的 弹 性 手 段', '众 趋 明 所 避 时 弃 道 犹 存', '桐 轩 潇 洒 远 尘 机 伯 始 英 风 世 不 衰 千 亩 自 封 轻 渭 叟 五 经 同 拜 重 宣 尼 衣 无 常 主 门 多 义 仓 有 余 粮 俗 共 熙 仁 智 眼 前 从 所 乐 利 名 身 外 更 何 思 琴 清 善 召 云 中 鹤 床 稳 高 蓍 叶 上 龟 茶 煮 玉 泉 僧 至 日 酒 𫇴 金 蚁 菊 芳 时 往 来 虚 席 宾 朋 盛 朝 暮 过 堂 子 弟 奇 万 卷 诗 书 归 腹 箧 地 仙 踪 迹 少 人 知']


In [44]:
print("去除符号后的训练结果：")
train_NBayes(x_train_pro, x_test_pro, y_train, y_test)
train_TfIdf(x_train_pro, x_test_pro, y_train, y_test)
train_nGram(x_train_pro, x_test_pro, y_train, y_test,2)

去除符号后的训练结果：
朴素贝叶斯得分：0.87075
TF-IDF得分：0.8285
ngram,n为2时得分：0.87325


## Pro+: 头尾加上`<start>` `<end>` 标记

In [52]:
train_data_prop = []
for paragraph in train_data_pro:
    train_data_prop.append("<start> "+paragraph+" <end>")

print(train_data_prop[6000])


<start> 作 者 程 苏 东 北 京 大 学 中 文 系 讲 师 北 京 1 0 0 8 7 1 <end>


In [55]:
x_train_prop, x_test_prop, y_train, y_test = train_test_split(train_data_prop, train_target,test_size=0.4,random_state=0)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(analyzer='char')
vector_prop = cv.fit_transform(x_train_prop)
vector_prop.todense()
vectorizer_prop = CountVectorizer(analyzer='char', max_features=5000)
vectorizer_prop.fit(x_train_prop)
print(x_train_prop[4000])

<start> 淡 云 残 雪 簇 江 天 策 蹇 迟 回 客 兴 阑 持 钵 老 僧 来 咒 水 倚 船 商 女 待 搬 滩 沙 翘 白 鹭 非 真 静 竹 映 繁 梅 奈 苦 寒 阮 籍 莫 嗟 歧 路 异 旧 山 溪 畔 有 渔 竿 <end>


In [56]:
print("加上标记后的训练结果：")
train_NBayes(x_train_prop, x_test_prop, y_train, y_test)
train_TfIdf(x_train_prop, x_test_prop, y_train, y_test)
train_nGram(x_train_prop, x_test_prop, y_train, y_test,2)

加上标记后的训练结果：
朴素贝叶斯得分：0.85975
TF-IDF得分：0.83775
ngram,n为2时得分：0.883
